Cell 1: Mount Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')
print("Google Drive mounted successfully!")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive mounted successfully!


Cell 2: Install and Import Libraries


In [2]:
# Install LightGBM
!pip install lightgbm scikit-learn pandas numpy joblib

# Import libraries
import pandas as pd
import numpy as np
import lightgbm as lgb
import joblib
import time
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import LabelEncoder


In [3]:
# Example CSV path in Google Drive
csv_path = "/content/drive/MyDrive/Colab Notebooks/CricketScorePrediction/Dataset/final_enhanced_dataset.csv"

# Load dataset
df = pd.read_csv(csv_path)
print("Dataset loaded. Shape:", df.shape)
df.head()


Dataset loaded. Shape: (906734, 34)


match_id        date          venue          series toss_winner  \
0    211028  2005-06-13  The Rose Bowl  Unknown Series     England   
1    211028  2005-06-13  The Rose Bowl  Unknown Series     England   
2    211028  2005-06-13  The Rose Bowl  Unknown Series     England   
3    211028  2005-06-13  The Rose Bowl  Unknown Series     England   
4    211028  2005-06-13  The Rose Bowl  Unknown Series     England   

  toss_decision               result match_format    team1      team2  ...  \
0           bat  England by 100 runs          T20  England  Australia  ...   
1           bat  England by 100 runs          T20  England  Australia  ...   
2           bat  England by 100 runs          T20  England  Australia  ...   
3           bat  England by 100 runs          T20  England  Australia  ...   
4           bat  England by 100 runs          T20  England  Australia  ...   

   bowler Run_In_Last5 Wickets_In_Last5  AverageScore  cumulative_runs  \
0   B Lee          0.0              0.0      1.278246                0   
1   B Lee          1.0              0.0      1.278246                1   
2   B Lee          1.0              0.0      1.278246                1   
3   B Lee          1.0              0.0      1.278246                1   
4   B Lee          1.0              0.0      1.278246                1   

   chasing_or_setting  target_score  required_run_rate  partnership_runs  \
0                   0           0.0                0.0                28   
1                   0         100.0                0.0                28   
2                   0          67.0                0.0                28   
3                   0          50.0                0.0                28   
4                   0          40.0                0.0                28   

   last_wicket_fall  
0               0.0  
1               0.0  
2               0.0  
3               0.0  
4               0.0  

[5 rows x 34 columns]

In [4]:
# Drop duplicates if they exist
df.drop_duplicates(inplace=True)

# Handle missing values - fill with 0 or drop rows, depending on your strategy
df.fillna(0, inplace=True)

# Convert 'date' column to datetime if needed
if 'date' in df.columns:
    df['date'] = pd.to_datetime(df['date'], errors='coerce').fillna(method='ffill')

print("After cleaning, dataset shape:", df.shape)
df.head()


After cleaning, dataset shape: (906734, 34)


<ipython-input-4-c264cb2d8af3>:9: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['date'] = pd.to_datetime(df['date'], errors='coerce').fillna(method='ffill')


match_id       date          venue          series toss_winner  \
0    211028 2005-06-13  The Rose Bowl  Unknown Series     England   
1    211028 2005-06-13  The Rose Bowl  Unknown Series     England   
2    211028 2005-06-13  The Rose Bowl  Unknown Series     England   
3    211028 2005-06-13  The Rose Bowl  Unknown Series     England   
4    211028 2005-06-13  The Rose Bowl  Unknown Series     England   

  toss_decision               result match_format    team1      team2  ...  \
0           bat  England by 100 runs          T20  England  Australia  ...   
1           bat  England by 100 runs          T20  England  Australia  ...   
2           bat  England by 100 runs          T20  England  Australia  ...   
3           bat  England by 100 runs          T20  England  Australia  ...   
4           bat  England by 100 runs          T20  England  Australia  ...   

   bowler Run_In_Last5 Wickets_In_Last5  AverageScore  cumulative_runs  \
0   B Lee          0.0              0.0      1.278246                0   
1   B Lee          1.0              0.0      1.278246                1   
2   B Lee          1.0              0.0      1.278246                1   
3   B Lee          1.0              0.0      1.278246                1   
4   B Lee          1.0              0.0      1.278246                1   

   chasing_or_setting  target_score  required_run_rate  partnership_runs  \
0                   0           0.0                0.0                28   
1                   0         100.0                0.0                28   
2                   0          67.0                0.0                28   
3                   0          50.0                0.0                28   
4                   0          40.0                0.0                28   

   last_wicket_fall  
0               0.0  
1               0.0  
2               0.0  
3               0.0  
4               0.0  

[5 rows x 34 columns]

In [5]:
# Example: If we need balls left in a T20 match (120 balls total per innings)
if 'over' in df.columns and 'ball' in df.columns:
    df['balls_left'] = 120 - (df['over'] * 6 + df['ball'])

# Wickets left: assuming 10 wickets in total
if 'wickets' in df.columns:
    df['wickets_left'] = 10 - df['wickets']

# If 'phase' is not numeric, we might encode it
# We'll do that in the next cell using LabelEncoder.

# Check your new columns
df.head()


match_id       date          venue          series toss_winner  \
0    211028 2005-06-13  The Rose Bowl  Unknown Series     England   
1    211028 2005-06-13  The Rose Bowl  Unknown Series     England   
2    211028 2005-06-13  The Rose Bowl  Unknown Series     England   
3    211028 2005-06-13  The Rose Bowl  Unknown Series     England   
4    211028 2005-06-13  The Rose Bowl  Unknown Series     England   

  toss_decision               result match_format    team1      team2  ...  \
0           bat  England by 100 runs          T20  England  Australia  ...   
1           bat  England by 100 runs          T20  England  Australia  ...   
2           bat  England by 100 runs          T20  England  Australia  ...   
3           bat  England by 100 runs          T20  England  Australia  ...   
4           bat  England by 100 runs          T20  England  Australia  ...   

   Wickets_In_Last5 AverageScore cumulative_runs  chasing_or_setting  \
0               0.0     1.278246               0                   0   
1               0.0     1.278246               1                   0   
2               0.0     1.278246               1                   0   
3               0.0     1.278246               1                   0   
4               0.0     1.278246               1                   0   

   target_score  required_run_rate  partnership_runs  last_wicket_fall  \
0           0.0                0.0                28               0.0   
1         100.0                0.0                28               0.0   
2          67.0                0.0                28               0.0   
3          50.0                0.0                28               0.0   
4          40.0                0.0                28               0.0   

   balls_left  wickets_left  
0         119            10  
1         118            10  
2         117            10  
3         116            10  
4         115            10  

[5 rows x 36 columns]

In [6]:
# Cell 6: Data Preparation
import pandas as pd
from sklearn.model_selection import train_test_split

# Ensure target column is defined correctly
target_column = "cumulative_runs"

# Define optimal features for training
features = [
    "venue", "team1", "team2", "over", "ball", "bowler",
    "Run_In_Last5", "Wickets_In_Last5", "AverageScore",
    "chasing_or_setting", "partnership_runs", "last_wicket_fall"
]

# Ensure target column exists
if target_column not in df.columns:
    raise ValueError(f"Target column '{target_column}' not found in dataset.")

# Select relevant columns
df_model = df[features + [target_column]].copy()

# Handle missing values if any
df_model = df_model.dropna()

# Encode categorical variables (LightGBM handles categorical features better)
for col in ["venue", "team1", "team2", "bowler", "chasing_or_setting"]:
    df_model[col] = df_model[col].astype("category")

# Split into train and test sets (80-20 split)
X = df_model[features]
y = df_model[target_column]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training Samples: {X_train.shape[0]}, Testing Samples: {X_test.shape[0]}")


Training Samples: 725387, Testing Samples: 181347


In [8]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split

# Split the training set further into training and validation subsets
X_train_sub, X_valid, y_train_sub, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Define categorical features explicitly
categorical_features = ["venue", "team1", "team2", "bowler", "chasing_or_setting"]

# Convert dataset into LightGBM format
train_data = lgb.Dataset(X_train_sub, label=y_train_sub, categorical_feature=categorical_features)
valid_data = lgb.Dataset(X_valid, label=y_valid, categorical_feature=categorical_features, reference=train_data)

# LightGBM Parameters (optimized for regression)
params = {
    "objective": "regression",
    "metric": "rmse",
    "boosting_type": "gbdt",
    "learning_rate": 0.05,
    "num_leaves": 31,
    "max_depth": -1,
    "feature_fraction": 0.8,
    "bagging_fraction": 0.8,
    "bagging_freq": 5,
    "verbosity": -1,
    "random_state": 42
}

# Train the model with early stopping using callback
lgb_model = lgb.train(
    params,
    train_data,
    num_boost_round=1000,
    valid_sets=[train_data, valid_data],
    callbacks=[lgb.early_stopping(stopping_rounds=50)]
)

print("✅ LightGBM Model Training Complete")


Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[1000]	training's rmse: 7.22184	valid_1's rmse: 7.99958
✅ LightGBM Model Training Complete


In [9]:
from sklearn.metrics import mean_absolute_error, r2_score

# Predict on test set
y_pred = lgb_model.predict(X_test, num_iteration=lgb_model.best_iteration)

# Calculate Metrics
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("\n📌 **Final Model Evaluation on Test Set**")
print(f"✅ Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"✅ Mean Absolute Error (MAE): {mae:.4f}")
print(f"✅ R² Score: {r2:.4f}")



📌 **Final Model Evaluation on Test Set**
✅ Root Mean Squared Error (RMSE): 7.9860
✅ Mean Absolute Error (MAE): 5.8203
✅ R² Score: 0.9675


In [11]:
import joblib
import os

# Define the save path in Google Drive
save_path = "/content/drive/MyDrive/Colab Notebooks/CricketScorePrediction/Model/LightGBM"

# Ensure the directory exists (creates it if not)
os.makedirs(save_path, exist_ok=True)

# Full path to save the model
model_filename = os.path.join(save_path, "lightgbm_t20_final_score.pkl")

# Save the model
joblib.dump(lgb_model, model_filename)

print(f"✅ Model saved successfully at: {model_filename}")


✅ Model saved successfully at: /content/drive/MyDrive/Colab Notebooks/CricketScorePrediction/Model/LightGBM/lightgbm_t20_final_score.pkl


In [12]:
import matplotlib.pyplot as plt
import base64
from io import BytesIO
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# Predict on test set using the trained LightGBM model
y_pred = lgb_model.predict(X_test, num_iteration=lgb_model.best_iteration)

# Calculate evaluation metrics
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")
print(f"R² Score: {r2:.4f}")

# Helper function to convert a Matplotlib figure to a base64 string
def fig_to_base64(fig):
    buf = BytesIO()
    fig.savefig(buf, format='png', bbox_inches='tight')
    plt.close(fig)
    buf.seek(0)
    return base64.b64encode(buf.getvalue()).decode('utf-8')

# Plot 1: Actual vs Predicted Final Score
fig1, ax1 = plt.subplots(figsize=(8,6))
ax1.scatter(y_test, y_pred, alpha=0.5, color='blue')
ax1.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='red', linestyle='--')
ax1.set_title('Actual vs Predicted Final Score')
ax1.set_xlabel('Actual Final Score')
ax1.set_ylabel('Predicted Final Score')
img1 = fig_to_base64(fig1)

# Plot 2: Distribution of Prediction Errors
errors = y_test - y_pred
fig2, ax2 = plt.subplots(figsize=(8,6))
ax2.hist(errors, bins=30, edgecolor='black', color='green')
ax2.set_title('Distribution of Prediction Errors')
ax2.set_xlabel('Prediction Error')
ax2.set_ylabel('Frequency')
img2 = fig_to_base64(fig2)

# Plot 3: Feature Importance Plot (Top 10)
fig3, ax3 = plt.subplots(figsize=(8,6))
lgb.plot_importance(lgb_model, max_num_features=10, ax=ax3)
ax3.set_title('Top 10 Feature Importances')
img3 = fig_to_base64(fig3)

# Build an HTML report string
html_report = f"""
<html>
<head>
    <title>T20 Final Score Prediction Model Evaluation Report</title>
    <style>
        body {{ font-family: Arial, sans-serif; margin: 20px; }}
        h1 {{ color: #2E86C1; }}
        h2 {{ color: #2874A6; }}
        table, th, td {{ border: 1px solid #dddddd; border-collapse: collapse; padding: 8px; }}
        th {{ background-color: #f2f2f2; }}
        img {{ margin: 10px 0; }}
    </style>
</head>
<body>
    <h1>T20 Final Score Prediction Model Evaluation Report</h1>
    <h2>Evaluation Metrics</h2>
    <table>
        <tr>
            <th>Metric</th>
            <th>Value</th>
        </tr>
        <tr>
            <td>RMSE</td>
            <td>{rmse:.4f}</td>
        </tr>
        <tr>
            <td>MAE</td>
            <td>{mae:.4f}</td>
        </tr>
        <tr>
            <td>R² Score</td>
            <td>{r2:.4f}</td>
        </tr>
    </table>

    <h2>Evaluation Plots</h2>
    <h3>Actual vs Predicted Final Score</h3>
    <img src="data:image/png;base64,{img1}" width="600">

    <h3>Distribution of Prediction Errors</h3>
    <img src="data:image/png;base64,{img2}" width="600">

    <h3>Top 10 Feature Importances</h3>
    <img src="data:image/png;base64,{img3}" width="600">

    <h2>Real-World Scenario Analysis</h2>
    <p>Below are sample predictions on 10 real-world scenarios drawn from the test set:</p>
    <table>
        <tr>
            <th>Scenario</th>
            <th>Actual Final Score</th>
            <th>Predicted Final Score</th>
            <th>Absolute Error</th>
        </tr>
"""

# Add 10 sample scenarios from the test set
for i in range(min(10, len(y_test))):
    actual = y_test.iloc[i]
    pred = y_pred[i]
    abs_err = abs(actual - pred)
    html_report += f"""
        <tr>
            <td>Scenario {i+1}</td>
            <td>{actual:.2f}</td>
            <td>{pred:.2f}</td>
            <td>{abs_err:.2f}</td>
        </tr>
    """

html_report += """
    </table>

    <h2>Conclusions</h2>
    <p>
        The model achieves a high R² score, indicating that it explains a large portion of the variance in final scores.
        The RMSE and MAE values suggest that on average, the prediction error is reasonably low.
        However, there is room for improvement in reducing the prediction error for certain cases.
    </p>
</body>
</html>
"""

# Save the HTML report in Google Drive
report_path = "/content/drive/MyDrive/Colab Notebooks/CricketScorePrediction/Model/LightGBM/model_evaluation_report.html"
with open(report_path, "w") as f:
    f.write(html_report)

print(f"✅ Evaluation report saved as HTML at: {report_path}")


RMSE: 7.9860
MAE: 5.8203
R² Score: 0.9675
✅ Evaluation report saved as HTML at: /content/drive/MyDrive/Colab Notebooks/CricketScorePrediction/Model/LightGBM/model_evaluation_report.html
